In [1]:
import numpy as np
import pandas as pd
import xarray as xr

from tqdm import tqdm

In [2]:
df = pd.read_csv('../../data/finland/sample_plots_uerra_cdf_features.csv')
df.head()

,plot_id,lon,lat,num_pine,vol_pine,num_spruce,vol_spruce,num_deciduous,vol_deciduous,number,...,uerra_consecutive_dry_days_index_per_time_period,uerra_number_of_cdd_periods_with_more_than_5days_per_time_period,uerra_consecutive_wet_days_index_per_time_period,uerra_number_of_cwd_periods_with_more_than_5days_per_time_period,uerra_precipitation_days_index_per_time_period,uerra_wet_days_index_per_time_period,uerra_highest_one_day_precipitation_amount_per_time_period,uerra_highest_five_day_precipitation_amount_per_time_period,uerra_number_of_5day_heavy_precipitation_periods_per_time_period,uerra_simple_daily_intensity_index_per_time_period
0,509711,23.760515,61.857391,629,239.3,79,27.6,118,11.3,825,...,44.0,211.0,13.0,34.0,1846.0,1846.0,35.612881,35.612881,0.0,5.187795
1,509712,23.776417,61.857257,668,129.7,118,16.5,393,28.6,1179,...,44.0,216.0,12.0,29.0,1783.0,1783.0,34.724686,34.724686,0.0,5.170337
2,509713,23.762006,61.854816,865,194.8,0,0.0,39,0.2,904,...,42.0,213.0,13.0,30.0,1822.0,1822.0,35.866657,35.866657,0.0,5.132607
3,509714,23.761818,61.852331,2358,130.1,236,4.4,472,6.4,3065,...,42.0,213.0,13.0,30.0,1822.0,1822.0,35.866657,35.866657,0.0,5.132607
4,509715,23.762492,61.850755,200,105.7,320,127.4,20,0.4,540,...,42.0,213.0,13.0,30.0,1822.0,1822.0,35.866657,35.866657,0.0,5.132607


Load ERA5 CDO indexes

In [3]:
ds_cfd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacfd.nc")
ds_fd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecafd.nc")
ds_csu = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacsu.nc")
ds_id = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecaid.nc")
ds_su = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecasu.nc")

ds_cdd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacdd.nc")
ds_cwd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacwd.nc")
ds_pd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecapd.nc")
ds_rr1 = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecarr1.nc")
ds_rx1day = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecarx1day.nc")
ds_rx5day = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecarx5day.nc")
ds_sdii = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecasdii.nc")

rasters = [ds_cfd, ds_fd, ds_csu, ds_id, ds_su, ds_cdd, ds_cwd, ds_pd, ds_rr1, ds_rx1day, ds_rx5day, ds_sdii]

In [4]:
results = {}
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    lon = row['lon']
    lat = row['lat']
    for ds in rasters:
        for v in ds.variables:
            if 'height' in ds.variables:
                heights = ds['height']
            times = ds['time']

            if v not in ["time", "longitude", "latitude", "inate.", "oordinate.", "height"]:
                if v not in results:
                    results[v] = []
                if 'height' in ds.variables:
                    values = ds.sel(time=times[0], height=heights[0], longitude=lon, latitude=lat, method='nearest')[v].values.tolist()
                else:
                    values = ds.sel(time=times[0], longitude=lon, latitude=lat, method='nearest')[v].values.tolist()
                results[v].append(values)

100%|██████████| 19876/19876 [21:02<00:00, 16.11it/s]


In [7]:
for k in results:
    print(k)
    df[f'era5_{k}'] = results[k]

number_of_cfd_periods_with_more_than_5days_per_time_period
frost_days_index_per_time_period
number_of_csu_periods_with_more_than_5days_per_time_period
ice_days_index_per_time_period
summer_days_index_per_time_period
consecutive_dry_days_index_per_time_period
number_of_cdd_periods_with_more_than_5days_per_time_period
consecutive_wet_days_index_per_time_period
number_of_cwd_periods_with_more_than_5days_per_time_period
precipitation_days_index_per_time_period
wet_days_index_per_time_period
highest_one_day_precipitation_amount_per_time_period
highest_five_day_precipitation_amount_per_time_period
number_of_5day_heavy_precipitation_periods_per_time_period
simple_daily_intensity_index_per_time_period


In [6]:
df.to_csv('../../data/finland/sample_plots_all_features.csv', index=False)